In [5]:
from freelancersdk.session import Session
from freelancersdk.resources.users.users import search_freelancers, get_users, get_user_by_id, get_portfolios
from freelancersdk.resources.users.exceptions import \
    UsersNotFoundException, PortfoliosNotFoundException
from freelancersdk.resources.users.helpers import (
    create_get_users_details_object, create_get_users_object
)
from freelancersdk.resources.projects.projects import get_jobs
from freelancersdk.resources.projects.exceptions import \
    JobsNotFoundException
import os
import pandas as pd
import json

# Freelancer User Data

- The api is limited to returning 100 rows only for user data
- Filter user_df by 'closed' == False to exclude any closed accounts
- Exclude columns in this  list

  exclude_cols = [
    'limited_account',
    'pool_ids',
    'enterprise_ids',
    'escrowcom_account_linked',
    'escrowcom_interaction_required',
    'enterprises',
    'oauth_password_credentials_allowed',
    'registration_completed',
    'is_profile_visible',
    'registration_date',
    'chosen_role',
    'role',
    'status',
    'timezone'
  ]

In [52]:
oauth_token='PaWtwT7yMjkaPXEbODfOfQLHuSSEYE'
url='https://www.freelancer.com'
search_keywords = ['engineer', 'data', 'design', 'web']

exclude_user_cols = [
    'limited_account',
    'pool_ids',
    'enterprise_ids',
    'escrowcom_account_linked',
    'escrowcom_interaction_required',
    'enterprises',
    'oauth_password_credentials_allowed',
    'registration_completed',
    'is_profile_visible',
    'registration_date',
    'chosen_role',
    'role',
    'status',
    'timezone'
]

In [33]:
def get_fln_session():
    # url = os.environ.get('FLN_URL')
    # oauth_token = os.environ.get('FLN_OAUTH_TOKEN')
    return Session(oauth_token=oauth_token, url=url)

session = get_fln_session()

In [48]:
def search_freelance_users(session, query):
    user_details = create_get_users_details_object(
        country=True,
        status=True,
        display_info=True,
        profile_description=True,
        qualifications=True,
        portfolio=True,
        jobs=True,
        reputation=True,
        corporate_accounts=True
    )
    try:
        result = search_freelancers(
            session=session,
            query=query,
            user_details=user_details,
            limit=100 # maximum results = 100
        )
    except UsersNotFoundException as e:
        print('Error message: {}'.format(e.message))
        print('Server response: {}'.format(e.error_code))
        return None
    else:
        return result

In [50]:
# Get a list of dataframes by searching keywords one by one
df_list = [pd.DataFrame.from_dict(search_freelance_users(session,keyword)['users']) for keyword in search_keywords]

In [51]:
# Concatenate list into one consolidated dataframe
user_df = pd.concat(df_list, ignore_index=True)

In [53]:
# Drop all excluded columns
final_user_df = user_df.drop(exclude_user_cols, axis=1)
final_user_df

,id,username,closed,reputation,jobs,profile_description,hourly_rate,display_name,tagline,location,primary_currency,qualifications,primary_language,portfolio_count,corporate,public_name,company
0,33481141,VisualStudio77,False,{'entire_history': {'overall': 4.7516860802426...,"[{'id': 42, 'name': 'Engineering', 'category':...",******A complete Building ...,50.0,American HomeEngineering,"US Licensed Architect, Structural Engineer- Pe...","{'country': {'name': 'Bangladesh', 'flag_url':...","{'id': 1, 'code': 'USD', 'sign': '$', 'name': ...","[{'id': 533, 'name': 'Preferred Freelancer Pro...",en,9,"{'status': 'active', 'founder_id': 62421482}",American HomeEngineering,Archi Engineering Studio
1,39824241,JaneBurke,False,{'entire_history': {'overall': 4.9887323659978...,"[{'id': 42, 'name': 'Engineering', 'category':...",I'm Neamat manager of a specialized Architect ...,50.0,JaneBurke,Licensed Architect/PE/ME team at USA+Canada+Aus.,"{'country': {'name': 'Bangladesh', 'flag_url':...","{'id': 1, 'code': 'USD', 'sign': '$', 'name': ...","[{'id': 533, 'name': 'Preferred Freelancer Pro...",en,16,NaN,Md Neamat Ullah H.,
2,27245644,DavidJG23,False,{'entire_history': {'overall': 4.9999999999999...,"[{'id': 44, 'name': 'Mobile App Development', ...",Welcome to my profile!\n\nI’m David Jimenez Gu...,50.0,DavidJG23,iOS Engineer,"{'country': {'name': 'Andorra', 'flag_url': '/...","{'id': 8, 'code': 'EUR', 'sign': '€', 'name': ...","[{'id': 160, 'name': 'US English - Level 3', '...",en,0,NaN,David J.,
3,34205968,dervish11,False,{'entire_history': {'overall': 4.8902325275643...,"[{'id': 20, 'name': 'Graphic Design', 'categor...",Architectural & Civil Engineer | Fusing Creati...,50.0,dervish11,Architectural & Engineering,"{'country': {'name': 'Albania', 'flag_url': '/...","{'id': 8, 'code': 'EUR', 'sign': '€', 'name': ...","[{'id': 51, 'name': 'AutoCAD 2010', 'level': 1...",en,39,NaN,Dervish M.,
4,31295818,furkancakir,False,{'entire_history': {'overall': 4.9827955402735...,"[{'id': 6, 'name': 'C Programming', 'category'...",Hi. I'm Senior Mechanical Design Engineer and ...,70.0,furkancakir,Senior Mechanical Design Engineer,"{'country': {'name': 'Turkey', 'flag_url': '/i...","{'id': 1, 'code': 'USD', 'sign': '$', 'name': ...","[{'id': 69, 'name': 'Freelancer Orientation', ...",en,10,NaN,Furkan Taha C.,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,17059938,joec184,False,{'entire_history': {'overall': 4.9253745828873...,"[{'id': 17, 'name': 'Website Design', 'categor...",laconic.\nadj. expressing much in few words\n\...,80.0,laconic,Branding | Packaging | PP | Web | 3D | VisualM...,"{'country': {'name': 'United Kingdom', 'flag_u...","{'id': 4, 'code': 'GBP', 'sign': '£', 'name': ...","[{'id': 533, 'name': 'Preferred Freelancer Pro...",en,12,"{'status': 'active', 'founder_id': 54726682}",laconic,laconic.
396,51841746,SEOIdeal,False,{'entire_history': {'overall': 4.9697077105673...,"[{'id': 17, 'name': 'Website Design', 'categor...","I am Minakshi Mewara, Digital Marketing Manage...",2.0,SEOIdeal,SEO - No #1 Ranking on Google & Organic Traffi...,"{'country': {'name': 'India', 'flag_url': '/im...","{'id': 11, 'code': 'INR', 'sign': '₹', 'name':...","[{'id': 105, 'name': 'UK English - Level 1', '...",en,21,"{'status': 'expired', 'founder_id': 57675827}",SEOIdeal,SEOIdeal
397,29037031,programmatis,False,{'entire_history': {'overall': 4.9998038816035...,"[{'id': 3, 'name': 'PHP', 'category': {'id': 1...",\n\nWelcome to Programmatis!\n\nYour Trusted D...,250.0,Programmatis,Your Trusted Digital Solutions Partner,"{'country': {'name': 'United States', 'flag_ur...","{'id': 1, 'code': 'USD', 'sign': '$', 'name': ...","[{'id': 532, 'name': 'CSS - Level 1', 'level':...",en,18,"{'status': 'active', 'founder_id': 9253246}",Programmatis,Programmatis
398,6779051,linkgit,False,{'entire_history': {'overall': 4.9441573652145...,"[{'id': 3, 'name': 'PHP', 'category': {'id': 1...",We are a team of 50+ SEO professionals and Wor...,11.0,Smart 

In [54]:
print(final_user_df.shape)
# Export csv
final_user_df.to_csv('data/raw/fln_users.csv', index=False)

(400, 17)


## Freelancers final_user_df data exploration

In [14]:
df.to_string(max_colwidth=None)

'         id          username  closed                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [8]:
user_ids = final_user_df['id'].tolist()

# Get Portfolio

In [44]:
def sample_get_portfolios(session, user_ids):
    user_ids = user_ids
    # 3112780
    try:
        portfolios = get_portfolios(session, user_ids=user_ids)
    except PortfoliosNotFoundException as e:
        print('Error message: {}'.format(e.message))
        print('Server: response: {}'.format(e.error_code))
        return None
    else:
        return portfolios

result = sample_get_portfolios(user_ids)

if result:
    portfolios = []
    users = result['users'].keys()
    for k in users:
        portfolios+=(result['portfolios'][k])
    print('Got {} portfolios from {} users.'.format(len(portfolios), len(users)))
    print('Portfolios:\n {}'.format(portfolios))

Got 84 portfolios from 10 users.
Portfolios:
 [{'id': 11140211, 'user_id': 118215, 'title': 'Internal Jobs Management App', 'description': 'Snow Queen App manages all dispatched job card assignments for our internal snow service crews. The app tracks active job cards as well as upcoming ones, it allows operators to set their availability dynamically via time-blocks. All completed job cards are stored and calculated in regard to estimated payout, and all data collected during job servicing is uploaded and stored for liability purposes.', 'content_type': 'picture', 'featured': False, 'files': [{'id': 20714323, 'file_id': 200715610, 'filename': 'snow_1.PNG', 'description': None, 'cdn_url': '//cdn2.f-cdn.com/files/download/200715610/snow_1.PNG', 'thumbnails': [{'portfolio_file_id': 20714323, 'file_id': 200715610, 'filename': 'snow_1.PNG', 'width': None, 'height': None, 'cdn_url': '//cdn2.f-cdn.com/files/download/200715610/snow_1.PNG'}, {'portfolio_file_id': 20714323, 'file_id': 200716889, 

In [45]:
df3 = pd.DataFrame.from_records(portfolios)

In [46]:
df3

,id,user_id,title,description,content_type,featured,files,articles,jobs,categories,last_modify_date,position
0,11140211,118215,Internal Jobs Management App,Snow Queen App manages all dispatched job card...,picture,False,"[{'id': 20714323, 'file_id': 200715610, 'filen...",[],[],[],1688619865,None
1,11138080,118215,Well-being & Affirmation App,This app provides users with a collection of p...,picture,False,"[{'id': 20708733, 'file_id': 200656216, 'filen...",[],[],[],1688555043,None
2,10039597,118215,True Date (Dating & Meeting Apps),So the project description starts with their u...,picture,False,"[{'id': 17949708, 'file_id': 170195319, 'filen...",[],"[44, 59, 1067]",[],1653390928,None
3,10039511,118215,Liquor Ordering App (Smugglers),"Smugglers let customers order wine, beer, liqu...",picture,False,"[{'id': 17949589, 'file_id': 170194549, 'filen...",[],"[44, 59, 1067]",[],1653388937,None
4,10039486,118215,WASSILA: (On Demand Services App),Application that connects customer and service...,picture,False,"[{'id': 17949509, 'file_id': 170193300, 'filen...",[],"[59, 1067]",[],1653387734,None
...,...,...,...,...,...,...,...,...,...,...,...,...
79,9892229,50501541,RETAINING WALL CALCULATIONS AND PLANS,In this project it was required to calculate ...,picture,False,"[{'id': 17577501, 'file_id': 166122241, 'filen...",[],"[186, 188, 883]",[],1648148158,None
80,9892215,50501541,RESIDENTIAL HOUSE REVIT MODELLING,In this project an architectural design was re...,picture,False,"[{'id': 17577478, 'file_id': 166122090, 'filen...",[],[186],[],1648147812,None
81,9572720,50501541,EDIFICIO MODULAR// STICKEN BUILDING,THIS IS A DESIGN FOR A 1 STOREY BUILDING CLOSE...,picture,False,"[{'id': 16814841, 'file_id': 157771476, 'filen...",[],"[186, 188, 193, 271]",[],1637355931,None
82,9572658,50501541,AGORA// AMPHITHEATRE,THIS IS AN AMPHITEATHRE DONE IN PARAISO TABSAS...,picture,False,"[{'id': 16814698, 'file_id': 157769903, 'filen...",[],"[186, 193, 271]",[],1637353383,None


In [47]:
df3.columns

Index(['id', 'user_id', 'title', 'description', 'content_type', 'featured',
       'files', 'articles', 'jobs', 'categories', 'last_modify_date',
       'position'],
      dtype='object')

In [48]:
df3[['id', 'user_id', 'title', 'description', 'jobs']]

,id,user_id,title,description,jobs
0,11140211,118215,Internal Jobs Management App,Snow Queen App manages all dispatched job card...,[]
1,11138080,118215,Well-being & Affirmation App,This app provides users with a collection of p...,[]
2,10039597,118215,True Date (Dating & Meeting Apps),So the project description starts with their u...,"[44, 59, 1067]"
3,10039511,118215,Liquor Ordering App (Smugglers),"Smugglers let customers order wine, beer, liqu...","[44, 59, 1067]"
4,10039486,118215,WASSILA: (On Demand Services App),Application that connects customer and service...,"[59, 1067]"
...,...,...,...,...,...
79,9892229,50501541,RETAINING WALL CALCULATIONS AND PLANS,In this project it was required to calculate ...,"[186, 188, 883]"
80,9892215,50501541,RESIDENTIAL HOUSE REVIT MODELLING,In this project an architectural design was re...,[186]
81,9572720,50501541,EDIFICIO MODULAR// STICKEN BUILDING,THIS IS A DESIGN FOR A 1 STOREY BUILDING CLOSE...,"[186, 188, 193, 271]"
82,9572658,50501541,AGORA// AMPHITHEATRE,THIS IS AN AMPHITEATHRE DONE IN PARAISO TABSAS...,"[186, 193, 271]"


In [49]:
df3['description'][0]

'Snow Queen App manages all dispatched job card assignments for our internal snow service crews. The app tracks active job cards as well as upcoming ones, it allows operators to set their availability dynamically via time-blocks. All completed job cards are stored and calculated in regard to estimated payout, and all data collected during job servicing is uploaded and stored for liability purposes.'

In [67]:
result = df3.query('id==9572720')

print(result.to_string(max_colwidth=None))

         id   user_id                                title                                                                                                                                                          description content_type  featured                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [70]:
def sample_get_jobs(session,job_ids):
    get_jobs_data = {
        'job_ids': job_ids,
        'seo_details': True,
        'lang': 'en',
    }
    try:
        j = get_jobs(session, **get_jobs_data)
    except JobsNotFoundException as e:
        print('Error message: {}'.format(e.message))
        print('Server response: {}'.format(e.error_code))
        return None
    else:
        return j


j = sample_get_jobs([186, 188, 193, 271])
if j:
    print('Found jobs: {}'.format(len(j)))

Found jobs: 4


In [72]:
df4 = pd.DataFrame.from_records(j)
df4

,id,name,category,active_project_count,seo_url,seo_info,local
0,186,Civil Engineering,"{'id': 5, 'name': 'Engineering & Science'}",None,civil_engineering,{'seo_text': '<p>Civil engineering a broad and...,False
1,188,Structural Engineering,"{'id': 5, 'name': 'Engineering & Science'}",None,structural_engineering,{'seo_text': '<p>Why hire a structural enginee...,False
2,193,Finite Element Analysis,"{'id': 5, 'name': 'Engineering & Science'}",None,finite_element_analysis,{'seo_text': '<p>Finite Element Analysis (FEA)...,False
3,271,AutoCAD,"{'id': 5, 'name': 'Engineering & Science'}",None,autocad,{'seo_text': '<p>AutoCAD is one of the most ub...,False


In [74]:
df4['seo_info'][0]

{'seo_text': "<p>Civil engineering a broad and expansive professional engineering discipline which is focused on designing, creating and maintaining the infrastructures all around us that bridge the connection between the human experience and the natural world just outside our perspective. Some typical civil engineering projects could be the construction of roadways, canals, bridges, culverts, dams. Civil engineering has a longstanding history in both the private and public sectors. It is practised by individuals, organizations, firms, projects may be sponsored by local, state, national or international governments, the military; the sky is really the limit when it comes to civil engineering projects. A very multidisciplinary focus in most cases, civil engineers are highly skilled at navigating the complexity and nuance of the technical requirements of their projects while also knowing how to work with the human systems and interfaces that are largely at the center of funding and admin